In [18]:
pip install tensorflow 

Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd 
import numpy as np
import tensorflow as tf 
from tensorflow.keras import layers,models
import tensorflowjs as tfjs
import joblib

C:\Users\Sumit\anaconda3\Lib\site-packages\tensorflowjs\read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [22]:
# data loading and preprocessing 
df = pd.read_csv("heart.csv")

In [ ]:
cholesterol_mean = df.loc[df['Cholesterol'] != 0, 'Cholesterol'].mean()
df['Cholesterol'] = df['Cholesterol'].replace(0, cholesterol_mean)
df['Cholesterol'] = df['Cholesterol'].round(2)

In [ ]:
resting_bp_mean = df.loc[df['RestingBP'] != 0, 'RestingBP'].mean()
df['RestingBP'] = df['RestingBP'].replace(0, resting_bp_mean)
df['RestingBP'] = df['RestingBP'].round(2)


In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded = df_encoded.astype(int)


In [ ]:
X = df_encoded.drop('HeartDisease', axis=1)
y = df_encoded['HeartDisease']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
num_features = X_train_scaled.shape[1]
model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(num_features,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Sigmoid activation for binary classification
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy', # Standard for binary classification
              metrics=['accuracy'])

In [ ]:
print("Training the new TensorFlow model...")
model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, verbose=1)


In [ ]:
keras_model_path = 're_trained_heart_model.h5'
model.save(keras_model_path)
print(f"Keras model saved to {keras_model_path}")

In [ ]:
tfjs_model_dir = 'tfjs_heart_model'
tfjs.converters.save_keras_model(model, tfjs_model_dir)
print(f"Model successfully converted to TensorFlow.js format and saved in the '{tfjs_model_dir}' directory.")


In [ ]:
joblib.dump(scaler, 'Scaler.pkl')
joblib.dump(X.columns.tolist(), 'Columns.pkl')
print("Scaler and column names saved for MERN stack use.")